In [1]:
!pip install transformers
!pip install sentencepiece
!pip install datasets
!pip install accelerate 
!pip install evaluate
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.0

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
#print(test_dataset['highlights'])

In [ ]:
import json
i=[0,0,0];
with open("train_social2.json",'r',encoding='utf-8') as train:
    data = json.load(train)
    article=[];
    highlights=[];
    for line in data:
        article.append(line[0]+"</s>"+line[1])
        if(line[2]==0):
            highlights.append('沒有關聯')
            i[0]+=1
        elif(line[2]==1):
            highlights.append('互相支持')
            i[1]+=1
        else:
            highlights.append('兩者衝突')
            i[2]+=1
    train_dataset = {'article':article,'highlights':highlights}
    print(i)


[667, 3505, 2046]


In [ ]:
with open("dev_social2.json",'r',encoding='utf-8') as dev:
    data = json.load(dev)
    article=[];
    highlights=[];
    for line in data:
        article.append(line[0]+"</s>"+line[1])
        if(line[2]==0):
            highlights.append('沒有關聯')
        elif(line[2]==1):
            highlights.append('互相支持')
        else:
            highlights.append('兩者衝突')
    val_dataset = {'article':article,'highlights':highlights}


In [ ]:
with open("test_social2.json",'r',encoding='utf-8') as test:
    data = json.load(test)
    article=[];
    highlights=[];
    for line in data:
        article.append(line[0]+"</s>"+line[1])
        if(line[2]==0):
            highlights.append('沒有關聯')
        elif(line[2]==1):
            highlights.append('互相支持')
        else:
            highlights.append('兩者衝突')
    test_dataset = {'article':article,'highlights':highlights}

In [ ]:
# -*- coding: utf-8 -*-
'''from datasets import load_dataset

dataset = load_dataset('cnn_dailymail', '3.0.0',data_dir="daily_mail")

dataset.keys()

len(dataset['test'])

train_dataset=dataset["train"][0:5000]
val_dataset=dataset["validation"][0:200]
test_dataset=dataset["test"][0:200]
print(train_dataset)'''

from transformers import  AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
tokenize_train_data=tokenizer(train_dataset['article'],text_target=train_dataset['highlights'],padding=True,truncation=True,max_length=512)
tokenize_val_data=tokenizer(val_dataset['article'],text_target=val_dataset['highlights'],padding=True,truncation=True,max_length=512)
tokenize_test_data=tokenizer(test_dataset['article'],text_target=test_dataset['highlights'],padding=True,truncation=True,max_length=512)
print(tokenize_train_data.keys())

import torch
from torch.utils.data import DataLoader
class cnnDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

train_data =cnnDataset(tokenize_train_data)
val_data =cnnDataset(tokenize_val_data)
test_data =cnnDataset(tokenize_test_data)

import logging
import datasets
from datasets import load_dataset, load_metric
from torch.utils.data import DataLoader
from tqdm.auto import tqdm, trange
import math

dict_keys(['input_ids', 'attention_mask', 'labels'])


In [ ]:
print(tokenize_train_data['input_ids'][0])
print(tokenizer.decode(tokenize_train_data['input_ids'][0]))
print(tokenize_train_data['labels'][0])
print(tokenizer.decode(tokenize_train_data['labels'][0]))

[259, 32030, 17225, 27510, 15915, 24552, 1193, 10691, 89441, 4057, 66446, 78849, 40857, 7823, 1374, 74485, 11153, 47392, 169369, 14296, 55834, 1, 259, 16746, 25780, 205764, 23035, 31875, 17481, 1083, 1290, 328, 18507, 259, 168705, 1290, 353, 17481, 2811, 33387, 17955, 259, 7609, 1637, 81318, 1374, 229967, 17481, 7738, 16746, 25780, 184355, 23035, 31875, 3862, 259, 21239, 21648, 111601, 108651, 1223, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
import transformers
from accelerate import Accelerator ,find_executable_batch_size
from transformers import (
    AdamW,
    AutoConfig,
    default_data_collator,
    get_scheduler
)

train_batch_size = 16      # 設定 training batch size
eval_batch_size = 16     # 設定 eval batch size
num_train_epochs = 100     # 設定 epoch

In [ ]:
data_collator = default_data_collator
train_dataloader = DataLoader(train_data, shuffle=True, collate_fn=data_collator, batch_size=train_batch_size)
eval_dataloader = DataLoader(val_data, collate_fn=data_collator, batch_size=eval_batch_size)

print(len(train_dataloader))
print(len(train_data))

from transformers import MT5ForConditionalGeneration, T5Tokenizer

model = MT5ForConditionalGeneration.from_pretrained("google/mt5-small")

learning_rate=1e-5          # 設定 learning_rate
gradient_accumulation_steps = 6   # 設定 幾步後進行反向傳播

no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)

# Scheduler and math around the number of training steps.
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / gradient_accumulation_steps)
max_train_steps = num_train_epochs * num_update_steps_per_epoch
print('max_train_steps', max_train_steps)

# scheduler
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=max_train_steps,
)

# Initialize the accelerator. We will let the accelerator handle device placement for us in this example.
accelerator = Accelerator()

# Prepare everything with our `accelerator`.
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

logger = logging.getLogger(__name__)
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger.info(accelerator.state)
output_dir = 'model_mt5/'


total_batch_size = train_batch_size * accelerator.num_processes * gradient_accumulation_steps

logger.info("***** Running training *****")
logger.info(f"  Num examples = {len(train_dataset)}")
logger.info(f"  Num Epochs = {num_train_epochs}")
logger.info(f"  Instantaneous batch size per device = {train_batch_size}")
logger.info(f"  Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
logger.info(f"  Gradient Accumulation steps = {gradient_accumulation_steps}")
logger.info(f"  Total optimization steps = {max_train_steps}")


completed_steps = 0
best_epoch = {"epoch": 0, "rouge": 0 }
best_epoch = {"epoch": 0, "acc": 0 }


389
6218


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


max_train_steps 6500


In [ ]:
metric = load_metric("accuracy")
print(metric)

<ipython-input-12-374824d292e7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


Metric(name: "accuracy", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions (`list` of `int`): Predicted labels.
    references (`list` of `int`): Ground truth labels.
    normalize (`boolean`): If set to False, returns the number of correctly classified samples. Otherwise, returns the fraction of correctly classified samples. Defaults to True.
    sample_weight (`list` of `float`): Sample weights Defaults to None.

Returns:
    accuracy (`float` or `int`): Accuracy score. Minimum possible value is 0. Maximum possible value is 1.0, or the number of examples input, if `normalize` is set to `True`.. A higher score means higher accuracy.

Examples:

    Example 1-A simple example
        >>> accuracy_metric = datasets.load_metric("accuracy")
        >>> results = accuracy_metric.compute(references=[0, 1, 2, 0, 1, 2], predictions=[0, 1, 1, 2, 1, 0])
        >>> print(results)
        {'accuracy': 0.5}

   

In [ ]:
import evaluate
rouge_score = evaluate.load("rouge")
for epoch in trange(num_train_epochs, desc="Epoch"):
    model.train()
    for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
    
        outputs = model(**batch)
        loss = outputs.loss
        loss = loss / gradient_accumulation_steps
        accelerator.backward(loss)
        if step % gradient_accumulation_steps == 0 or step == len(train_dataloader) - 1:
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            completed_steps += 1

        if step % 100 == 0:
            print({'epoch': epoch, 'step': step, 'loss': loss.item()})

        if completed_steps >= max_train_steps:
            break

    logger.info("***** Running eval *****")
    model.eval()
    preds=[]
    answers=[]
    with torch.no_grad():
        for step, batch in enumerate(tqdm(eval_dataloader, desc="Eval Iteration")):
            outputs = model(**batch)
            output_ids = model.generate(batch['input_ids'], num_beams=2, max_length=5, early_stopping=True)
            preds.append(" ".join([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in output_ids]))
            #print(output_ids)
            answers.append(" ".join([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in batch['labels']]))
            predictions = output_ids[0]
            #print(batch['labels'])
            try:
                metric.add_batch(predictions=output_ids[0],references=batch['labels'][0])
            except:
                print("1")
    '''
    scores = rouge_score.compute(
    predictions=preds, references=answers
    )
    print(scores)
    eval_precision=scores["rougeL"]
    logger.info(f"epoch :{epoch} \n rougeL:  {eval_precision}")
    if eval_precision > best_epoch["rouge"]:
        best_epoch['epoch'] = num_train_epochs
        best_epoch['rouge'] = eval_precision
    ///

    for step, batch in enumerate(tqdm(eval_dataloader, desc="Eval Iteration")):
    outputs = model(**batch)
    '''

    eval_metric = metric.compute()
    logger.info(f"epoch {epoch}: {eval_metric}")
    print(eval_metric['accuracy'])
    if eval_metric['accuracy'] > best_epoch['acc']:
        best_epoch['epoch'] = num_train_epochs #記錄最佳的epoch
        best_epoch['acc'] = eval_metric['accuracy'] #記錄最佳的準確率




    if(epoch%5==0 or best_epoch['epoch'] == epoch):
        unwrapped_model = accelerator.unwrap_model(model)
        unwrapped_model.save_pretrained(output_dir + 'new_epoch_' + str(epoch) + '/', save_function=accelerator.save)

    """if output_dir is not None:
        accelerator.wait_for_everyone()
        unwrapped_model = accelerator.unwrap_model(model)
        unwrapped_model.save_pretrained(output_dir + 'new_epoch_' + str(epoch) + '/', save_function=accelerator.save)
"""
print(best_epoch)





Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

Iteration:   0%|          | 0/389 [00:00<?, ?it/s]

{'epoch': 0, 'step': 0, 'loss': 2.747450351715088}
{'epoch': 0, 'step': 100, 'loss': 2.707331657409668}
{'epoch': 0, 'step': 200, 'loss': 2.305471420288086}
{'epoch': 0, 'step': 300, 'loss': 2.5280508995056152}


Eval Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

Iteration:   0%|          | 0/389 [00:00<?, ?it/s]

...

1
1
1
1
1
1
1
1


Iteration:   0%|          | 0/389 [00:00<?, ?it/s]

{'epoch': 5, 'step': 0, 'loss': 2.490081787109375}
{'epoch': 5, 'step': 100, 'loss': 1.8703391551971436}
{'epoch': 5, 'step': 200, 'loss': 1.8804659843444824}
{'epoch': 5, 'step': 300, 'loss': 1.7603211402893066}


Eval Iteration:   0%|          | 0/51 [00:00<?, ?it/s]

1
1
1
1


In [ ]:
print(len(preds))

51


In [ ]:
print(preds)
print()

['<extra_id_0>支持 <extra_id_0>支持支持 <extra_id_0>支持 <extra_id_0>支持 <extra_id_0>支持 <extra_id_0> <extra_id_0>立場立場立場 <extra_id_0>立場支持 <extra_id_0>支持 <extra_id_0>支持 <extra_id_0> <extra_id_0>支持 <extra_id_0>支持 <extra_id_0> <extra_id_0>支持 <extra_id_0>支持', '<extra_id_0>支持 <extra_id_0>立場立場 <extra_id_0>支持 <extra_id_0>立場立場立場 <extra_id_0> <extra_id_0>支持 <extra_id_0>支持 <extra_id_0>支持 <extra_id_0>立場立場立場 <extra_id_0>立場立場立場 <extra_id_0> <extra_id_0> <extra_id_0>立場支持 <extra_id_0>支持 <extra_id_0>支持 <extra_id_0>支持', '<extra_id_0>立場立場立場 <extra_id_0>立場 <extra_id_0>立場立場立場 <extra_id_0>支持 <extra_id_0>支持 <extra_id_0>立場 <extra_id_0>立場立場立場 <extra_id_0> <extra_id_0>立場支持 <extra_id_0>立場立場立場 <extra_id_0>支持 <extra_id_0>支持 <extra_id_0>立場立場立場 <extra_id_0>立場立場 <extra_id_0>立場 <extra_id_0>支持', '<extra_id_0>立場 <extra_id_0>支持 <extra_id_0>立場 <extra_id_0>立場立場 <extra_id_0>立場立場立場 <extra_id_0>立場支持 <extra_id_0> <extra_id_0>立場立場立場 <extra_id_0>支持 <extra_id_0>支持 <extra_id_0>立場立場立場 <extra_id_0>支持 <extra_id_0>支持 <extra_id_0>支持 <extra_id_0

In [ ]:
from transformers import AutoConfig, BertForSequenceClassification, default_data_collator
from torch.utils.data import DataLoader
from accelerate import Accelerator
from tqdm.auto import tqdm


In [ ]:
#cd drive

In [ ]:
#ls

In [ ]:
config = AutoConfig.from_pretrained("./model_mt5/new_epoch_24/config.json") 
model = MT5ForConditionalGeneration.from_pretrained("./model_mt5/new_epoch_24/pytorch_model.bin", config = config,).to(device)

In [ ]:
def C_model(model, context, question):

    input_encodings = tokenizer([context],truncation=True, padding=True,max_length=512)
    input_dataset = cnnDataset(input_encodings)
    #print("hi")
    data_collator = default_data_collator
    input_dataloader = DataLoader(input_dataset, collate_fn=data_collator, batch_size=1)  
    #print("hi")
    accelerator = Accelerator()
    model, input_dataloader = accelerator.prepare(model, input_dataloader)
    for batch in input_dataloader:
        output_ids = model.generate(**batch, num_beams=3, max_length=8, early_stopping=True)
        predictions_text = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in output_ids]

    return predictions_text

In [ ]:
cnt=0
errorcnt=0
answer=[]
for i in range(len(test_dataset['article'])):
    cnt+=1
    context = test_dataset['article'][i]
    question = 'What is the relation?'
    answer = C_model(model, context, question)
    print(answer)
    #print(type(answer[0]))
    #k=input()
    #if answer()!=test_dataset['highlights'][i]:
    #    errorcnt+=1
#print(predict)      
print(cnt)
print(errorcnt)
print(1-(errorcnt/cnt))

['<extra_id_0>支持']
['<extra_id_0>支持']
['<extra_id_0>立場支持']
['<extra_id_0>支持']
['<extra_id_0>支持']
['<extra_id_0>支持']
['<extra_id_0>支持']
['<extra_id_0>支持']
['<extra_id_0>支持']
['<extra_id_0>立場立場']
['<extra_id_0>立場立場立場立場立場立場']
['<extra_id_0>立場支持']
['<extra_id_0>支持']
['<extra_id_0>支持']
['<extra_id_0>支持']
['<extra_id_0>支持']
['<extra_id_0>支持']
['<extra_id_0>支持']
['<extra_id_0>立場支持']
['<extra_id_0>支持']
['<extra_id_0>立場支持']
['<extra_id_0>立場立場立場立場立場']
['<extra_id_0>立場支持']
['<extra_id_0>支持支持支持支持']
['<extra_id_0>立場立場立場立場立場']
['<extra_id_0>']
['<extra_id_0>支持']
['<extra_id_0>立場']
['<extra_id_0>立場支持']
['<extra_id_0>支持']
['<extra_id_0>立場支持']
['<extra_id_0>立場立場']
['<extra_id_0>支持']
['<extra_id_0>立場']
['<extra_id_0>支持']
['<extra_id_0>立場支持']
['<extra_id_0>支持支持']
['<extra_id_0>']
['<extra_id_0>立場支持']
['<extra_id_0>支持']
['<extra_id_0>']
['<extra_id_0>支持']
['<extra_id_0>支持']
['<extra_id_0>支持']
['<extra_id_0>立場支持']
['<extra_id_0>支持']
['<extra_id_0>立場立場立場立場立場立場']
['<extra_id_0>立場支持']
['<extra_id_0>支持']
['<ex

In [ ]:
from sklearn.metrics import classification_report
print("no support=0. support=1")
print(classification_report(testlabel,answer))